In [3]:
import sys
sys.path.append('../')
sys.path.append('../scripts')
import os
import pandas as pd
from transformers import AutoTokenizer,AutoModelForSequenceClassification, DistilBertTokenizer, AdamW
from sklearn.model_selection import train_test_split
from scripts.train import train_and_evaluate_model
from scripts.data_loader import CryptoTwittDataset
from transformers import RobertaTokenizer,RobertaModel
import torch
from scripts.data_handler import DataOrganize, DataModes

In [11]:
cache_dir_d_drive = "D:/models"
model_name = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 3)
data_organize = DataOrganize()
data_organize.create_clean_data(mode=DataModes.RANDOM_1000)
review_texts = data_organize.data["body"]
filtered_data = data_organize.data[data_organize.data['entities.sentiment.basic'].notna()]
label_mapping = {"Bullish": 2, "Bearish": 0}

# Apply the mapping to convert string labels to integers
filtered_data['labels'] = filtered_data['entities.sentiment.basic'].map(label_mapping)
review_texts = filtered_data["body"].tolist()
labels = filtered_data['labels'].tolist()

review_texts_train, review_texts_test, labels_train, labels_test = train_test_split(review_texts, labels, test_size=0.2, random_state=42)


train_loader, val_loader = CryptoTwittDataset.getDataLoader(review_texts_train, review_texts_test, labels_train, labels_test,tokenizer)
fine_tuned_model = train_and_evaluate_model(train_loader, val_loader, model)
# train_and_evaluate_model(preprocessor.X_train_preprocessed, preprocessor.y_train_preprocessed, preprocessor.X_test_preprocessed, preprocessor.y_test_preprocessed)

# fine_tuned_model.save_pretrained("fine_tuned_distilroberta_base")
torch.save(model.state_dict(), "../models/fine_tuned_Cryptobert.pth")

100%|██████████| 1000/1000 [00:15<00:00, 65.84it/s]


<class 'str'>


100%|██████████| 724/724 [00:00<00:00, 17467.22it/s]
C:\Users\Asus\AppData\Local\Temp/ipykernel_16272/1891792980.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['labels'] = filtered_data['entities.sentiment.basic'].map(label_mapping)
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
100%|██████████| 19/19 [03:51<00:00, 12.21s/it]


Epoch 1, Training Loss: 0.3127094805240631, Validation Loss: 0.3886119514703751, Mean Difference: 0.18421052631578946


100%|██████████| 19/19 [03:47<00:00, 11.95s/it]


Epoch 2, Training Loss: 0.26414909958839417, Validation Loss: 0.29942330420017244, Mean Difference: 0.15789473684210525


100%|██████████| 19/19 [03:57<00:00, 12.52s/it]


Epoch 3, Training Loss: 0.01935632713139057, Validation Loss: 0.40348033979535103, Mean Difference: 0.2894736842105263


RuntimeError: Parent directory ../models does not exist.